## Librerías y rutas

In [1]:
import os
import pandas as pd

In [2]:
path = './Estaciones'

## Función para leer los datos de la estación, regresa un diccionario

In [13]:
def leer_smn_encabezado( archivo ):
    
    with open( archivo, 'rb' ) as f:
   
        next( f ).decode('latin-1')
        next( f ).decode('latin-1')
        next( f ).decode('latin-1')
        next( f ).decode('latin-1')
   
        est_numero = next( f ).decode('latin-1')
        est_numero = int( est_numero.split(':')[1] )
        est_nombre = next( f ).decode('latin-1')
        est_nombre = est_nombre.split(':')[1].strip()
        est_estado = next( f ).decode('latin-1')
        est_estado = est_estado.split(':')[1].strip()
        est_municipio = next( f ).decode('latin-1')
        est_municipio = est_municipio.split(':')[1].strip()
        est_situacion = next( f ).decode('latin-1')
        est_organismo = next( f ).decode('latin-1')
        est_cve = next( f ).decode('latin-1')
        est_latitud = next( f ).decode('latin-1')
        est_latitud = est_latitud.split(':')[1].strip()
        est_latitud = est_latitud.split('°')[0]       #elimina "°"
        est_longitud = next( f ).decode('latin-1')
        est_longitud = est_longitud.split(':')[1].strip()
        est_longitud = est_longitud.split('°')[0]     #elimina "°"
        est_altitud = next( f ).decode('latin-1')
        est_altitud = est_altitud.split(':')[1].strip()
        est_altitud = est_altitud.split(' ')[0].replace(',','')       #elimina " msnm"
   
        print( est_numero, est_nombre, est_estado, est_municipio )
        print( est_latitud, est_longitud, est_altitud )
        
#        fest.write(str(est_numero)+", "+ est_latitud+", "+ est_longitud+ ", "+ est_altitud+"\n")
#        fest.write(str(est_numero)+", "+ est_latitud+", "+ est_longitud +"\n")
    
        encabezado = { 'numero': est_numero,
                       'nombre': est_nombre,
                       'estado': est_estado,
                       'municipio': est_municipio,
                       'situacion': est_situacion,
                       'organismo': est_organismo,
                       'cve': est_cve,
                       'latitud': est_latitud,
                       'longitud': est_longitud,
                       'altitud': est_altitud }
        return encabezado
    

## Extraer las coordenadas de todos los archivos y guardarlas en "estaciones.csv"

In [14]:
fest = open(path+"/estaciones.csv",'w')
fest.write("Numero, Latitud, Longitud, Altitud\n")

26

In [15]:
files = os.listdir(path)

for ff in files:
    if ff.endswith(".txt"):
        archivo = os.path.join( path, ff )
        enc = leer_smn_encabezado( archivo )
        fest.write(str(enc['numero'])+", "+ enc['latitud']+", "+ enc['longitud']+ ", "+ enc['altitud']+"\n")

fest.close()

print("terminado el proceso")

7175 VILLAFLORES (SMN) CHIAPAS VILLAFLORES
016.229 -093.263 554
13022 PACHUCA (OBS) HIDALGO PACHUCA DE SOTO
020.088 -098.750 2369
14006 AJOJUCAR JALISCO TEOCALTICHE
021.563 -102.439 1745
32038 NIEVES ZACATECAS GENERAL FRANCISCO R. MURGUIA
024.001 -103.019 1913
14142 TAPALPA (SMN) JALISCO TAPALPA
019.945 -103.758 2060
5026 PRESA COYOTE COAHUILA DE ZARAGOZA TORREON
025.542 -103.469 1223
14068 HOSTOTIPAQUILLO JALISCO HOSTOTIPAQUILLO
021.058 -104.051 1300
3062 SANTIAGO BAJA CALIFORNIA SUR LOS CABOS
023.482 -109.718 132
3038 MULEGE BAJA CALIFORNIA SUR MULEGE
026.889 -111.984 10
8099 MAJALCA CHIHUAHUA CHIHUAHUA
028.803 -106.485 2119
12002 ACAPULCO DE JUAREZ (OBS) GUERRERO ACAPULCO DE JUAREZ
016.763 -099.749 3
14030 CIUDAD GUZMAN (OBS) JALISCO ZAPOTLAN EL GRANDE
019.718 -103.465 1515
14141 INGENIO TAMAZULA JALISCO TAMAZULA DE GORDIANO
019.688 -103.258 1450
11079 VALLE DE SANTIAGO GUANAJUATO VALLE DE SANTIAGO
020.383 -101.179 1790
32054 SOMBRERETE (DGE) ZACATECAS SOMBRERETE
023.635 -103.640 23

## Función para leer los datos, regresa un data frame

In [6]:
def leer_smn_mediciones(archivo):
    
    est_numero = archivo.split('.')[1].split('/')[2]
    
    archivosalida=archivo.split('.')[0]+".csv"
 
    print("archivo de entrada: "+archivo)
    print("archivo de salida: "+archivosalida)
       
    df = pd.read_fwf( archivo, skiprows=18 )
   
    df.columns = [ 'Fecha', 'Precipitacion', 'Evaporacion', 'T_max', 'T_min' ]
    print( 'Dataframe original... ', df.shape )

    #    df = df[ df['Precipitacion'].str[0]!='-' ] # sí funciona
    #    df = df[ df['Precipitacion'].apply( lambda x: '-' not in x) ] # sí funciona
    mask = df['Precipitacion'].str.contains( r'-' )
    df = df[ ~mask ]
    print( 'Revisado - en precipitacion... ', df.shape )

    mask = df['Fecha'].str.contains( r'\d\d/\d\d/\d\d\d\d', na=False )
    df = df[ mask ]
    print( 'Revisada la fecha... ', df.shape )
   
    df['Precipitacion'] = df['Precipitacion'].apply(pd.to_numeric, errors='coerce')
    df['Evaporacion'] = df['Evaporacion'].apply(pd.to_numeric, errors='coerce')
    df['T_max'] = df['T_max'].apply(pd.to_numeric, errors='coerce')
    df['T_min'] = df['T_min'].apply(pd.to_numeric, errors='coerce')
   

    #Versión para buscar errores
    # try:    
    #     df['Dia'] = df['Fecha'].apply( lambda x: x.split('/')[0] )
    #     df['Mes'] = df['Fecha'].apply( lambda x: x.split('/')[1] )
    #     df['Anyo'] = df['Fecha'].apply( lambda x: x.split('/')[2] )
    # except Exception as e:
    #     print( e.__class__ )
    #     for index, row in df.iterrows():
    #         fecha = row['Fecha']
    #         try:
    #             dma = row['Fecha'].split('/')
    #             if len( dma )<3:
    #                 print( dma )
    #         except Exception as e:
    #             print( dma, type(dma) )                
    #             print( e.__class__ )
    #             return 0

    df['Dia'] = df['Fecha'].apply( lambda x: x.split('/')[0] )
    df['Mes'] = df['Fecha'].apply( lambda x: x.split('/')[1] )
    df['Anyo'] = df['Fecha'].apply( lambda x: x.split('/')[2] )

    df['Estacion'] = est_numero
    df = df[ [ 'Estacion', 'Dia', 'Mes', 'Anyo', 'Precipitacion', 'Evaporacion', 'T_max', 'T_min' ] ]
    
    return df
           
#    df.to_csv( archivosalida, sep=',', index=None, na_rep=-99999 )

## Se leen todos los archivos, pero solo los datos, y se concatenan en un solo archivo grande

In [7]:
files = os.listdir(path)
files = [ os.path.join( path, ff ) for ff in files if 'txt' in ff]

In [8]:
df_grande = pd.concat( [ leer_smn_mediciones(ff) for ff in files ] )

print( df_grande.shape )

print("terminado el proceso")

archivo de entrada: ./Estaciones/7175.txt
archivo de salida: .csv
Dataframe original...  (30104, 5)
Revisado - en precipitacion...  (30103, 5)
Revisada la fecha...  (30103, 5)
archivo de entrada: ./Estaciones/13022.txt
archivo de salida: .csv
Dataframe original...  (26793, 5)
Revisado - en precipitacion...  (26792, 5)
Revisada la fecha...  (26792, 5)
archivo de entrada: ./Estaciones/14006.txt
archivo de salida: .csv
Dataframe original...  (28020, 5)
Revisado - en precipitacion...  (28019, 5)
Revisada la fecha...  (28019, 5)
archivo de entrada: ./Estaciones/32038.txt
archivo de salida: .csv
Dataframe original...  (27244, 5)
Revisado - en precipitacion...  (27243, 5)
Revisada la fecha...  (27243, 5)
archivo de entrada: ./Estaciones/14142.txt
archivo de salida: .csv
Dataframe original...  (27528, 5)
Revisado - en precipitacion...  (27527, 5)
Revisada la fecha...  (27527, 5)
archivo de entrada: ./Estaciones/5026.txt
archivo de salida: .csv
Dataframe original...  (27259, 5)
Revisado - en pr

archivo de entrada: ./Estaciones/17001.txt
archivo de salida: .csv
Dataframe original...  (31833, 5)
Revisado - en precipitacion...  (31832, 5)
Revisada la fecha...  (31832, 5)
archivo de entrada: ./Estaciones/11096.txt
archivo de salida: .csv
Dataframe original...  (27153, 5)
Revisado - en precipitacion...  (27152, 5)
Revisada la fecha...  (27152, 5)
archivo de entrada: ./Estaciones/13012.txt
archivo de salida: .csv
Dataframe original...  (38064, 5)
Revisado - en precipitacion...  (38063, 5)
Revisada la fecha...  (38063, 5)
archivo de entrada: ./Estaciones/13042.txt
archivo de salida: .csv
Dataframe original...  (27986, 5)
Revisado - en precipitacion...  (27985, 5)
Revisada la fecha...  (27985, 5)
archivo de entrada: ./Estaciones/16051.txt
archivo de salida: .csv
Dataframe original...  (27249, 5)
Revisado - en precipitacion...  (27248, 5)
Revisada la fecha...  (27248, 5)
archivo de entrada: ./Estaciones/3008.txt
archivo de salida: .csv
Dataframe original...  (29088, 5)
Revisado - en p

archivo de entrada: ./Estaciones/14084.txt
archivo de salida: .csv
Dataframe original...  (27490, 5)
Revisado - en precipitacion...  (27489, 5)
Revisada la fecha...  (27489, 5)
archivo de entrada: ./Estaciones/8246.txt
archivo de salida: .csv
Dataframe original...  (30449, 5)
Revisado - en precipitacion...  (30448, 5)
Revisada la fecha...  (30448, 5)
archivo de entrada: ./Estaciones/16124.txt
archivo de salida: .csv
Dataframe original...  (27093, 5)
Revisado - en precipitacion...  (27092, 5)
Revisada la fecha...  (27092, 5)
archivo de entrada: ./Estaciones/30125.txt
archivo de salida: .csv
Dataframe original...  (27131, 5)
Revisado - en precipitacion...  (27130, 5)
Revisada la fecha...  (27130, 5)
archivo de entrada: ./Estaciones/3035.txt
archivo de salida: .csv
Dataframe original...  (27682, 5)
Revisado - en precipitacion...  (27681, 5)
Revisada la fecha...  (27681, 5)
archivo de entrada: ./Estaciones/3012.txt
archivo de salida: .csv
Dataframe original...  (27978, 5)
Revisado - en pre

In [9]:
df_grande.to_csv( os.path.join( path, 'smn_mediciones.csv' ), index=False )

In [12]:
df_grande.to_parquet( os.path.join( path, 'smn_mediciones.parquet' ), index=False )